In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

## 0. 학습 준비
- 데이터셋 로드
- 환경 설정 (패키지 설치 및 로드)

In [24]:
import os

# === 입출력 경로 ===
os.environ.setdefault("CSV_PATH", "labeled_sentences_voted_every_data.csv")   # seed 파일 경로

# 배치 단위 (한 번에 몇 개의 항목을 생성시킬지)
os.environ.setdefault("BATCH_SIZE", "10")

# 생성 다양성
os.environ.setdefault("TEMPERATURE", "0.8")
os.environ.setdefault("TOP_P", "0.9")

print("환경설정 완료")


환경설정 완료


In [68]:

# 환경 설정과 데이터 로드
import re, random, json, types
import numpy as np
import pandas as pd
from pathlib import Path

CSV_PATH = os.getenv("CSV_PATH")

df = pd.read_csv(CSV_PATH)
df_daily = pd.read_csv(CSV_PATH)
df_biz = pd.read_csv("schedule_dataset_augmented_epoch_2.csv")
df_daily = df_daily.drop(columns=["file_path"])
df_biz = df_biz[df_biz["label"] != 0]
print("Loaded shape:", df_daily.shape)
print("Loaded shape:", df_biz.shape)
display(df_daily.head())
display(df_biz.head())

Loaded shape: (50000, 5)
Loaded shape: (1260, 8)


,sentence,label_pred,agree_rate,certainty_avg,accept
0,전화받았습니다 #@소속#입니다,0,1.0,83,True
1,그것도 해주시고 다 해주세요 왜냐면 저도 이제 몇 번을 참은 거라서요,0,1.0,75,True
2,네 다른 건의사항 있으십니까,0,1.0,85,True
3,아 그래요 #@기타#문고나 #@기타#문고에도 없어요,0,1.0,76,True
4,감사합니다 고객님 #@소속# 고객센터였습니다,0,1.0,100,True


,sentence,domain,task,label,confidence,source,seed_id,epoch
1,오늘 밤에 색감 체크 가능하신가요?,design_production,CREATE,1,0.92,teacher,1.0,0
3,금요일 오후 킥오프 미팅 할까요?,design_production,CREATE,1,0.95,teacher,3.0,0
5,채색 일정 이번 주로 당길 수 있을까요?,design_production,UPDATE,1,0.89,teacher,5.0,0
7,잉킹 리뷰 내일 오전이면 괜찮으실까요?,design_production,CREATE,1,0.91,teacher,7.0,0
8,오늘 회의 그냥 패스하죠,design_production,CANCEL,1,0.94,teacher,8.0,0


In [69]:
 
# 텍스트/라벨 컬럼 자동 감지 및 분할
from sklearn.model_selection import train_test_split

text_col = "sentence" if "sentence" in df.columns else df.columns[0]
label_col = "label_pred" if "label_pred" in df.columns else df.columns[-1]
print("Using columns -> text:", text_col, "label:", label_col)

df = df[[text_col, label_col]].dropna().reset_index(drop=True)
X = df[text_col].astype(str).values
y = df[label_col].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=38, stratify=y
)
len(X_train), len(X_test)

Using columns -> text: sentence label: label_pred


(40000, 10000)

## TF-IDF 분류기

In [70]:

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

# 1) char ngram 기반
tfidf_char = Pipeline([
    ("tfidf", TfidfVectorizer(analyzer="char", ngram_range=(2,5), min_df=3)),
    ("clf", LinearSVC(class_weight="balanced"))
])
tfidf_char.fit(X_train, y_train)
pred_char = tfidf_char.predict(X_test)
acc_char = accuracy_score(y_test, pred_char)
print("TF‑IDF char Accuracy:", acc_char)
print(classification_report(y_test, pred_char))

# 2) word 기반
tfidf_word = Pipeline([
    ("tfidf", TfidfVectorizer(analyzer="word", token_pattern=r"(?u)\b\w+\b", min_df=3)),
    ("clf", LinearSVC(class_weight="balanced"))
])
tfidf_word.fit(X_train, y_train)
pred_word = tfidf_word.predict(X_test)
acc_word = accuracy_score(y_test, pred_word)
print("TF‑IDF word Accuracy:", acc_word)
print(classification_report(y_test, pred_word))

TF‑IDF char Accuracy: 0.9701
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      9417
           1       0.74      0.75      0.75       583

    accuracy                           0.97     10000
   macro avg       0.86      0.87      0.86     10000
weighted avg       0.97      0.97      0.97     10000

TF‑IDF word Accuracy: 0.9279
              precision    recall  f1-score   support

           0       0.98      0.94      0.96      9417
           1       0.43      0.68      0.52       583

    accuracy                           0.93     10000
   macro avg       0.70      0.81      0.74     10000
weighted avg       0.95      0.93      0.94     10000



In [71]:
import pandas as pd

df_labeled = pd.read_csv("schedule_dataset_augmented_epoch_2.csv")
display(df_labeled.head())

,sentence,domain,task,label,confidence,source,seed_id,epoch
0,배경 러프 수정본 여기 있어요,design_production,NONE,0,0.99,teacher,0.0,0
1,오늘 밤에 색감 체크 가능하신가요?,design_production,CREATE,1,0.92,teacher,1.0,0
2,싱크표 다시 올려드릴게요,design_production,NONE,0,0.98,teacher,2.0,0
3,금요일 오후 킥오프 미팅 할까요?,design_production,CREATE,1,0.95,teacher,3.0,0
4,오늘 콘티 피드백만 주세요,design_production,NONE,0,0.97,teacher,4.0,0


In [79]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support

# X, y 설정
X_test = df_labeled["sentence"]
y_test = df_labeled["label"]   # teacher 모델로 라벨링된 결과
display(X_test)
display(y_test.count())

# --- TF-IDF char 모델 ---
pred_char = tfidf_char.predict(X_test)
acc_char = accuracy_score(y_test, pred_char)
print("\n=== TF-IDF (char) 결과 ===")
print("Accuracy:", acc_char)
print("Confusion Matrix:\n", confusion_matrix(y_test, pred_char))
print(classification_report(y_test, pred_char, digits=3))

# --- TF-IDF word 모델 ---
pred_word = tfidf_word.predict(X_test)
acc_word = accuracy_score(y_test, pred_word)
print("\n=== TF-IDF (word) 결과 ===")
print("Accuracy:", acc_word)
print("Confusion Matrix:\n", confusion_matrix(y_test, pred_word))
print(classification_report(y_test, pred_word, digits=3))


0                                 배경 러프 수정본 여기 있어요
1                              오늘 밤에 색감 체크 가능하신가요?
2                                    싱크표 다시 올려드릴게요
3                               금요일 오후 킥오프 미팅 할까요?
4                                   오늘 콘티 피드백만 주세요
                           ...                    
1995              오늘 1시 채색팀 시간 맞아? 혹시 15분 늦춰도 괜찮아?
1996    내일 바빠서 채색 세션 못 하겠어... 뭐 할까? 혹시 다른 시간이 있나요?
1997                      콘티 초안 좀 더 빨리 받을 수 있나요? 🙃
1998      금요일 리뷰 시간, 15분 늦춰도 괜찮아? 내일 아침에 다시 조율해볼게.
1999                           오늘 콘티 피드백 좀 주실래요? 👀
Name: sentence, Length: 2000, dtype: object

0       0
1       1
2       0
3       1
4       0
       ..
1995    1
1996    1
1997    0
1998    1
1999    0
Name: label, Length: 2000, dtype: int64


=== TF-IDF (char) 결과 ===
Accuracy: 0.807
Confusion Matrix:
 [[ 550  190]
 [ 196 1064]]
              precision    recall  f1-score   support

           0      0.737     0.743     0.740       740
           1      0.848     0.844     0.846      1260

    accuracy                          0.807      2000
   macro avg      0.793     0.794     0.793      2000
weighted avg      0.807     0.807     0.807      2000


=== TF-IDF (word) 결과 ===
Accuracy: 0.7805
Confusion Matrix:
 [[ 502  238]
 [ 201 1059]]
              precision    recall  f1-score   support

           0      0.714     0.678     0.696       740
           1      0.816     0.840     0.828      1260

    accuracy                          0.780      2000
   macro avg      0.765     0.759     0.762      2000
weighted avg      0.779     0.780     0.779      2000



In [74]:
import numpy as np
from sklearn.metrics import precision_recall_curve, precision_score, recall_score, f1_score, average_precision_score

def pick_thr_for_precision(y_true, scores, target_p=0.90):
    p, r, th = precision_recall_curve(y_true, scores)
    cands = [(pp, rr, tt) for pp, rr, tt in zip(p[:-1], r[:-1], th) if pp >= target_p]
    if cands:
        # precision 만족 중 recall 최대
        best = max(cands, key=lambda x: x[1])
        return best[2], {"P": best[0], "R": best[1], "picked": f"P≥{target_p}"}
    # 없으면 F1 최대 지점
    f1s = []
    for pp, rr, tt in zip(p[:-1], r[:-1], th):
        f1 = 0 if (pp+rr)==0 else 2*pp*rr/(pp+rr)
        f1s.append((f1, tt, pp, rr))
    f1s.sort(reverse=True, key=lambda x: x[0])
    f1, thr, pp, rr = f1s[0]
    return thr, {"P": pp, "R": rr, "F1": f1, "picked": "maxF1"}

def eval_with_thr(model, X, y, target_p=0.90, name="model"):
    scores = model.decision_function(X)  # LinearSVC 점수
    thr, info = pick_thr_for_precision(y, scores, target_p)
    y_pred = (scores >= thr).astype(int)
    P = precision_score(y, y_pred, zero_division=0)
    R = recall_score(y, y_pred, zero_division=0)
    F1 = f1_score(y, y_pred, zero_division=0)
    AP = average_precision_score(y, scores)
    print(f"[{name}] thr={thr:.4f} ({info['picked']}) | VAL-like P={info['P']:.3f}, R={info['R']:.3f}")
    print(f"TEST-ish  P={P:.3f} R={R:.3f} F1={F1:.3f} PR-AUC={AP:.3f}")
    return thr, (P,R,F1,AP)

# df_labeled 전체가 테스트셋이라면 그대로 적용 (혹은 내부에서 80/20 쪼개 임계값 고르고 동일 분포에 평가)
thr_char, _ = eval_with_thr(tfidf_char, X_test, y_test, target_p=0.95, name="TF-IDF char")
thr_word, _ = eval_with_thr(tfidf_word, X_test, y_test, target_p=0.95, name="TF-IDF word")


[TF-IDF char] thr=1.4743 (P≥0.95) | VAL-like P=0.950, R=0.317
TEST-ish  P=0.950 R=0.317 F1=0.475 PR-AUC=0.912
[TF-IDF word] thr=2.3566 (P≥0.95) | VAL-like P=0.950, R=0.167
TEST-ish  P=0.950 R=0.167 F1=0.284 PR-AUC=0.873


In [75]:
from sklearn.calibration import CalibratedClassifierCV
cal_char = CalibratedClassifierCV(tfidf_char, method="sigmoid", cv=3).fit(X_train, y_train)
probs = cal_char.predict_proba(X_test)[:,1]
# → precision 목표에 맞춰 threshold 선택

In [76]:
import numpy as np
from sklearn.metrics import (
    precision_recall_curve, precision_score, recall_score, f1_score,
    average_precision_score, confusion_matrix, classification_report
)

def pick_thr_for_precision(y_true, probs, target_p=0.90):
    p, r, th = precision_recall_curve(y_true, probs)
    # thresholds 길이는 p,r보다 1 작음
    cands = [(pp, rr, tt) for pp, rr, tt in zip(p[:-1], r[:-1], th) if pp >= target_p]
    if cands:
        # precision 만족 중 recall 최대 지점
        best = max(cands, key=lambda x: x[1])
        return best[2], {"picked_by": f"P≥{target_p}", "P": best[0], "R": best[1]}
    # fallback: F1 최대 지점
    f1s = []
    for pp, rr, tt in zip(p[:-1], r[:-1], th):
        f1 = 0 if (pp+rr)==0 else 2*pp*rr/(pp+rr)
        f1s.append((f1, tt, pp, rr))
    f1s.sort(reverse=True, key=lambda x: x[0])
    f1, thr, pp, rr = f1s[0]
    return thr, {"picked_by": "maxF1", "P": pp, "R": rr, "F1": f1}

# 1) 임계값 선택 (예: Precision 0.90 목표)
thr, info = pick_thr_for_precision(y_test, probs, target_p=0.9)
print(f"[Threshold] {info['picked_by']} -> thr={thr:.4f}  (VAL-like P={info['P']:.3f}, R={info['R']:.3f})")

# 2) 테스트셋 예측 및 지표
y_pred = (probs >= thr).astype(int)

P = precision_score(y_test, y_pred, zero_division=0)
R = recall_score(y_test, y_pred, zero_division=0)
F1 = f1_score(y_test, y_pred, zero_division=0)
AP = average_precision_score(y_test, probs)  # PR-AUC은 확률 기반
cm = confusion_matrix(y_test, y_pred, labels=[0,1])

print(f"[TEST] Precision={P:.3f}  Recall={R:.3f}  F1={F1:.3f}  PR-AUC={AP:.3f}")
print("Confusion Matrix [rows=true 0/1, cols=pred 0/1]:\n", cm)
print("Classification report:\n", classification_report(y_test, y_pred, digits=3))


[Threshold] P≥0.9 -> thr=0.6493  (VAL-like P=0.900, R=0.652)
[TEST] Precision=0.900  Recall=0.652  F1=0.756  PR-AUC=0.908
Confusion Matrix [rows=true 0/1, cols=pred 0/1]:
 [[649  91]
 [439 821]]
Classification report:
               precision    recall  f1-score   support

           0      0.597     0.877     0.710       740
           1      0.900     0.652     0.756      1260

    accuracy                          0.735      2000
   macro avg      0.748     0.764     0.733      2000
weighted avg      0.788     0.735     0.739      2000



In [57]:
 # 텍스트/라벨 컬럼 자동 감지 및 분할
from sklearn.model_selection import train_test_split

text_col = "sentence" if "sentence" in df.columns else df.columns[0]
label_col = "label" if "label" in df.columns else df.columns[-1]
print("Using columns -> text:", text_col, "label:", label_col)

df = df[[text_col, label_col]].dropna().reset_index(drop=True)
X_test = df[text_col].astype(str).values
Y_test = df[label_col].values

print(X_test, Y_test)

Using columns -> text: sentence label: label_pred
['전화받았습니다 #@소속#입니다' '그것도 해주시고 다 해주세요 왜냐면 저도 이제 몇 번을 참은 거라서요'
 '네 다른 건의사항 있으십니까' ... '예 우선 센터에서 하게되면요' '열린펜션민박이요 한번 찾아봐야겠네요'
 '네 말씀 주시면 안내 드리겠습니다'] [0 0 0 ... 0 0 0]


In [23]:
# 3) 기존 데이터셋 평가

import joblib
from sklearn.metrics import accuracy_score, classification_report
model = joblib.load("artifacts/char_calibrated.joblib")

pred_char = model.predict(X_test)
acc_char = accuracy_score(Y_test, pred_char)
print("TF‑IDF char Accuracy:", acc_char)
print(classification_report(Y_test, pred_char))

TF‑IDF char Accuracy: 0.29228
              precision    recall  f1-score   support

       False       0.03      0.51      0.05      1830
        True       0.94      0.28      0.44     48170

    accuracy                           0.29     50000
   macro avg       0.48      0.40      0.24     50000
weighted avg       0.90      0.29      0.42     50000



## 모델 내보내기

In [27]:
import joblib, os

os.makedirs("artifacts", exist_ok=True)
joblib.dump(cal_char, "artifacts/char_calibrated.joblib")
print("✅ 모델 저장 완료:", os.path.abspath("artifacts/char_calibrated.joblib"))

✅ 모델 저장 완료: /home/j-k13s101/artifacts/char_calibrated.joblib
